In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# 指定當前的工作資料夾
import os
# 此處為google drive中的檔案路徑,drive為之前指定的工作根目錄，要加上
os.chdir("drive/Colab Notebooks") 

In [0]:
os.chdir("tit") 

In [0]:
ls

In [0]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance

In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [0]:
def clean_data(titanic):
    titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
    # child
    titanic["child"] = titanic["Age"].apply(lambda x: 1 if x < 15 else 0)

    # sex
    titanic["sex"] = titanic["Sex"].apply(lambda x: 1 if x == "male" else 0)

    titanic["Embarked"] = titanic["Embarked"].fillna("S")
    # embark
    def getEmbark(Embarked):
        if Embarked == "S":
            return 1
        elif Embarked == "C":
            return 2
        else:
            return 3
    titanic["embark"] = titanic["Embarked"].apply(getEmbark)

    # familysize
    titanic["fimalysize"] = titanic["SibSp"] + titanic["Parch"] + 1

    # cabin
    def getCabin(cabin):
        if cabin == "N":
            return 0
        else:
            return 1
    titanic["cabin"] = titanic["Cabin"].apply(getCabin)

    # name
    def getName(name):
        if "Mr" in str(name):
            return 1
        elif "Mrs" in str(name):
            return 2
        else:
            return 0
    titanic["name"] = titanic["Name"].apply(getName)

    titanic["Fare"] = titanic["Fare"].fillna(titanic["Fare"].median())

    return titanic
train_data = clean_data(train)
test_data = clean_data(test)

In [0]:
train_data.to_csv("train_xgb.csv", index=False)
test_data.to_csv("test_xgb.csv", index=False)

In [0]:
train_da = pd.read_csv("train_xgb.csv")

In [0]:
X_train = train_da[["Pclass", "sex", "child", "fimalysize", "Fare", "embark", "cabin"]]

In [0]:
y_train = train_data['Survived']

In [0]:
clf =XGBClassifier(learning_rate=0.1, max_depth=2, 
silent=True, objective='binary:logistic')

In [0]:
param_test = {
    'n_estimators': range(30, 50, 2),
    'max_depth': range(2, 7, 1)
}
grid_search = GridSearchCV(estimator = clf, param_grid = param_test, 
scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train)
grid_search.best_params_

In [0]:
model = xgb.XGBClassifier(max_depth=6, n_estimators=32).fit(X_train, y_train)

In [0]:
# 顯現重要Columns
fig, ax = plt.subplots(figsize=(5,5))
plot_importance(model, ax=ax)

In [0]:
testdata = pd.read_csv('test_xgb.csv')

In [0]:
X_test = testdata[["Pclass", "sex", "child", "fimalysize", "Fare", "embark", "cabin"]]

In [0]:
predictions = model.predict(X_test)

In [0]:
summit = pd.read_csv('submit.csv')

In [0]:
submission = pd.DataFrame({ 'PassengerId': summit['PassengerId'],
                            'Survived': predictions })
submission.to_csv("6.csv", index=False)